In [1]:
import os
if os.getcwd().split('/')[-1] == "notebooks":
    os.chdir('..')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import Module
import torch.optim as optim

from src.model import teacherNet, studentNet
from src.kd import distillation_loss, extract_logits
from src.train import train

from tqdm.notebook import tqdm
batch_size = 128

In [1]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data/', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





../torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


NameError: name 'torch' is not defined

In [10]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/cifar_kd_exp_1')

### Train Teacher

In [11]:
model = teacherNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train(model, trainloader, testloader, optimizer, criterion, epochs=10, writer=writer)

RuntimeError: shape '[-1, 784]' is invalid for input of size 12288

### Extract Logits

In [6]:
kd_targets = extract_logits(model, trainloader)
kdtrain = torch.utils.data.TensorDataset(trainset.data.float()/255.0, trainset.targets, kd_targets)
kdloader = torch.utils.data.DataLoader(kdtrain, batch_size=128, shuffle=False, num_workers=0)

### Train Student

In [7]:
model = studentNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train(model, kdloader, testloader, optimizer, distillation_loss, epochs=10, writer=writer)

/Users/ozubair/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


Epoch 1 accuracy = 92.07%
Epoch 2 accuracy = 94.26%
Epoch 3 accuracy = 95.39%
Epoch 4 accuracy = 95.90%
Epoch 5 accuracy = 96.44%
Epoch 6 accuracy = 96.70%
Epoch 7 accuracy = 96.98%
Epoch 8 accuracy = 97.11%
Epoch 9 accuracy = 97.21%
Epoch 10 accuracy = 97.17%

